In [14]:
# Importações necessárias
import json
from braket.aws import AwsSession, AwsDevice, AwsQuantumTask
from braket.circuits import Circuit
from boto3 import Session

# 1. Configurar a sessão AWS
# Substitua 'braket' pelo nome do seu perfil AWS
boto_session = Session(profile_name='default')
aws_session = AwsSession(boto_session=boto_session)

# 2. Definir o hardware e o circuito
# Vamos usar o simulador SV1, que é rápido para testes
# device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
device_arn = f"arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1"
device = AwsDevice(device_arn, aws_session=aws_session)


In [11]:

# Criar o circuito
# Este circuito simples de 3 qubits gera todas as 8 combinações de 0s e 1s
num_qubits = 3
circuit = Circuit()
for i in range(num_qubits):
    circuit.h(i)
for i in range(num_qubits):
    circuit.measure(i)

# 3. Executar o circuito
# Vamos usar 100 shots para ter uma boa quantidade de resultados
shots = 100
print(f"Executando o job quântico no simulador '{device.name}' com {shots} shots...")
task = device.run(circuit, shots=shots)


Executando o job quântico no simulador 'Aria 1' com 100 shots...


In [15]:
print(AwsQuantumTask(task.id).state())


QUEUED


In [12]:

# 4. Obter o resultado e inspecionar a resposta
print(f"Aguardando a conclusão da tarefa (ARN: {task.id})...")
result = task.result()

print("\n--- Resultado da Tarefa ---")
print(result)


Aguardando a conclusão da tarefa (ARN: arn:aws:braket:us-east-1:709381776175:quantum-task/5c39c1f3-a5f9-412f-b930-3d4b6d6542d6)...


KeyboardInterrupt: 

Task is in terminal state CANCELLED and no result is available.


In [6]:

# 5. Analisar o 'measurement_counts'
# Esta é a parte que você usará para gerar as jogadas
counts = result.measurement_counts
print("\n--- Measurement Counts ---")
print(counts)

# 6. Converter os resultados para coordenadas (exemplo)
# Supondo que você queira um tabuleiro 2x2.
# Você precisa de 1 qubit para x (0 ou 1) e 1 para y.
num_bits_x = 1
num_bits_y = 1

def binary_to_cartesian(binary_str):
    if len(binary_str) < num_bits_x + num_bits_y:
        return None, None
    x_binary = binary_str[:num_bits_x]
    y_binary = binary_str[num_bits_x:num_bits_x + num_bits_y]
    x = int(x_binary, 2)
    y = int(y_binary, 2)
    return x, y

print("\n--- Jogadas Geradas ---")
jogadas_geradas = []
for binary_string, count in counts.items():
    x, y = binary_to_cartesian(binary_string)
    if x is not None and y is not None:
        jogadas_geradas.append({
            "bit_string": binary_string,
            "coordenada": {"x": x, "y": y},
            "ocorrencias": count
        })

print(json.dumps(jogadas_geradas, indent=2))


--- Measurement Counts ---
Counter({'101': 17, '000': 17, '110': 15, '111': 15, '010': 12, '100': 9, '011': 8, '001': 7})

--- Jogadas Geradas ---
[
  {
    "bit_string": "101",
    "coordenada": {
      "x": 1,
      "y": 0
    },
    "ocorrencias": 17
  },
  {
    "bit_string": "110",
    "coordenada": {
      "x": 1,
      "y": 1
    },
    "ocorrencias": 15
  },
  {
    "bit_string": "001",
    "coordenada": {
      "x": 0,
      "y": 0
    },
    "ocorrencias": 7
  },
  {
    "bit_string": "010",
    "coordenada": {
      "x": 0,
      "y": 1
    },
    "ocorrencias": 12
  },
  {
    "bit_string": "100",
    "coordenada": {
      "x": 1,
      "y": 0
    },
    "ocorrencias": 9
  },
  {
    "bit_string": "111",
    "coordenada": {
      "x": 1,
      "y": 1
    },
    "ocorrencias": 15
  },
  {
    "bit_string": "011",
    "coordenada": {
      "x": 0,
      "y": 1
    },
    "ocorrencias": 8
  },
  {
    "bit_string": "000",
    "coordenada": {
      "x": 0,
      "y": 0
    },
